In [2]:
%%bash
pip install git+https://github.com/huggingface/transformers
pip install flash-attn


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vwxpb2u3
  Resolved https://github.com/huggingface/transformers to commit a7f29523361b2cc12e51c1f5133d95f122f6f45c
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 120.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=253780426 sha256=4e2f9e39313266b1544b68138b15b91ee6221eccf14f7902b7c6620351340810
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully bui

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vwxpb2u3


In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch

# Load the model on the available device(s) with default attention.
# Removed attn_implementation="flash_attention_2" as it was causing an ImportError.
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct", dtype="bfloat16", device_map="auto"
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=12)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


Loading weights:   0%|          | 0/625 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.22 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.93 GiB is free. Process 47401 has 11.81 GiB memory in use. Of the allocated memory 11.65 GiB is allocated by PyTorch, and 37.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct", dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-2B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


https://drfirst.tistory.com/entry/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EB%AA%A8%EB%8D%B8qwen2-VL%EB%A1%9C-%EB%8F%99%EC%98%81%EC%83%81-%EB%B6%84%EC%84%9D%ED%95%98%EA%B8%B0-GPU%EC%95%BC

https://github.com/QwenLM/Qwen3-VL

In [2]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")


# Messages containing a video url and a text query
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/home/bongo/porter_notebook/llm/qwen/boat.mp4",
            },
            {"type": "text", "text": "Describe this video."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
출처: https://drfirst.tistory.com/entry/오픈소스모델qwen2-VL로-동영상-분석하기-GPU야 [일등박사의 연구소:티스토리]

SyntaxError: invalid syntax (ipython-input-1151447910.py, line 41)

In [1]:
# 1. Install necessary libraries
!pip install -q git+https://github.com/huggingface/transformers accelerate bitsandbytes qwen-vl-utils

from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# 2. Load Model in 4-bit (Drastically reduces VRAM usage)
# We use 'Qwen2-VL' class as Qwen3 generally uses the same architecture/class structure in HF
# If your specific repo requires Qwen3 classes, ensure your transformers version supports it.
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", # or "Qwen/Qwen3-VL-2B-Instruct" if available
    torch_dtype=torch.float16,     # Use float16 for T4 GPU (bfloat16 is for Ampere/A100)
    device_map="auto",
    load_in_4bit=True,             # <--- Key fix: Loads model in 4-bit
    quantization_config=None,
)

# 3. Load Processor with Resolution Limits
# Limiting pixels prevents the 7GB memory spike during attention
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    min_pixels=256*28*28,
    max_pixels=1024*28*28 # Limits resolution to save memory
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Inference
# We use torch.no_grad to save even more memory
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 24.4 MB/s eta 0:00:00


config.json: 0.00B [00:00, ?B/s]

You are using a model of type qwen2_5_vl to instantiate a model of type qwen2_vl. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

TypeError: Qwen2VLForConditionalGeneration.__init__() got an unexpected keyword argument 'load_in_4bit'

In [1]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# 1. Define the 4-bit configuration explicitly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2. Load Model using AutoModel (Handles the class mismatch warning automatically)
model = AutoModelForVision2Seq.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    quantization_config=bnb_config, # <--- Correct way to pass 4-bit config
    device_map="auto",
)

# 3. Load Processor
# min/max_pixels help prevent OutOfMemory errors on large images
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    min_pixels=256*28*28,
    max_pixels=1024*28*28
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 4. Prepare inputs
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# 5. Generate
# Using torch.no_grad() is essential for saving memory
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2307: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading weights:   0%|          | 0/824 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The image depicts a serene beach scene with a woman and her dog. The woman is sitting on the sandy beach, facing away from the camera, and smiling at her golden retriever. The dog is sitting next to her, looking up at her with its front paws extended towards her hand. The beach is sandy, and the ocean waves can be seen in the background, creating a calm and peaceful atmosphere. The lighting suggests it might be either early morning or late afternoon, as the sun casts a warm glow over the entire scene.


In [ ]:
# You don't need to reload the model/processor. Just run this:

new_messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                # Replace this URL with any image URL you want
                "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/560px-PNG_transparency_demonstration_1.png",
            },
            # You can change the prompt here
            {"type": "text", "text": "What is in this picture? Be specific."},
        ],
    }
]

# Process inputs
text = processor.apply_chat_template(
    new_messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(new_messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# Generate
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)

# Decode output
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text[0])

In [1]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# 1. Define the 4-bit configuration explicitly
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2. Load Model using AutoModel (Handles the class mismatch warning automatically)
model = AutoModelForVision2Seq.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct",
    quantization_config=bnb_config, # <--- Correct way to pass 4-bit config
    device_map="auto",
)

# 3. Load Processor
# min/max_pixels help prevent OutOfMemory errors on large images
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct",
    min_pixels=256*28*28,
    max_pixels=1024*28*28
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 4. Prepare inputs
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# 5. Generate
# Using torch.no_grad() is essential for saving memory
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2307: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading weights:   0%|          | 0/625 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


This is a heartwarming photograph of a woman and her dog sharing a joyful moment on a sandy beach during sunset.

The scene is bathed in the warm, golden light of the setting sun, which casts a soft glow over the entire landscape. The woman, with long dark hair, is sitting on the sand, wearing a plaid shirt and dark pants. She is smiling and looking at her dog with a happy expression as they both extend their hands to give each other a high-five. The dog, a light-colored Labrador Retriever, is sitting with its front paws raised, and it is wearing a colorful harness with a


In [1]:
import torch
from transformers import AutoModelForImageTextToText, AutoProcessor, BitsAndBytesConfig # <--- Updated Import
from qwen_vl_utils import process_vision_info

# 1. Define the 4-bit configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2. Load Model using the new Class Name
model = AutoModelForImageTextToText.from_pretrained( # <--- Updated Class
    "Qwen/Qwen3-VL-2B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",
)

# ... (The rest of your processor and generation code remains exactly the same)

Loading weights:   0%|          | 0/625 [00:00<?, ?it/s]

In [2]:
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct",
    min_pixels=256*28*28,
    max_pixels=1024*28*28
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 4. Prepare inputs
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(model.device)

# 5. Generate
# Using torch.no_grad() is essential for saving memory
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=128)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


This is a heartwarming photograph of a woman and her dog sharing a joyful moment on a sandy beach at sunset.

The image is centered on a woman and a large, light-colored dog, likely a Labrador Retriever, sitting together on the sand. The woman, with long dark hair, is wearing a plaid shirt and is smiling as she playfully interacts with her dog. They are both looking at each other, and their hands are touching in a playful "high five" pose. The dog is sitting with its front paws raised, and it is wearing a colorful harness with a pattern of red, blue, and green
